In [1]:
import numpy as np
import pandas as pd
from numba import njit

In [2]:
df = pd.read_csv('../data/BTCUSDT_1h_2020-2025.csv')

In [3]:
@njit
def calculate_linear_regression_slopes(prices: np.ndarray, window: int) -> np.ndarray:

    n = len(prices)
    slopes = np.zeros(n)
    time_indices = np.arange(window)
    
    for i in range(window - 1, n):
        price_window = prices[i-window+1:i+1]
        time_mean = (window - 1) / 2
        price_mean = np.mean(price_window)
        
        # Calculate slope using least squares method
        numerator = np.sum((time_indices - time_mean) * (price_window - price_mean))
        denominator = np.sum((time_indices - time_mean) ** 2)
        slopes[i] = numerator / denominator if denominator != 0 else 0
    
    return slopes

def find_regimes(df: pd.DataFrame, regression_window: int = 10) -> pd.DataFrame:

    slopes = calculate_linear_regression_slopes(df['Close'].values, regression_window)
    df['Slope'] = slopes
    
    slope_volatility = pd.Series(slopes).rolling(regression_window).std().values
    df['Regime'] = 'Trend'
    df.loc[abs(df['Slope']) < slope_volatility, 'Regime'] = 'Range'
    
    return df

df_regimes = find_regimes(df, regression_window = 20)

In [4]:

df.set_index('Timestamp', inplace=True)

In [5]:
def plot_regimes_and_price(df, last_n_candles=None):
    import plotly.graph_objects as go

    if last_n_candles:
        df = df.tail(last_n_candles).copy()
    
    fig = go.Figure()
    
    # Find regime changes first
    df['regime_change'] = df['Regime'] != df['Regime'].shift(1)
    change_points = df[df['regime_change']].index.tolist()
    
    if change_points:
        # Add the start and end of the dataset
        change_points = [df.index[0]] + change_points + [df.index[-1]]
        
        # Create full-height regime bands
        for i in range(len(change_points)-1):
            start = change_points[i]
            end = change_points[i+1]
            regime = df.loc[start:end, 'Regime'].iloc[0]
            
            color = 'rgba(255,0,0,0.15)' if regime == 'Range' else 'rgba(0,255,0,0.15)'
            
            # Add rectangular shape for full height
            fig.add_shape(
                type="rect",
                x0=start,
                x1=end,
                y0=df['Low'].min() * 0.99,  # Slightly below lowest price
                y1=df['High'].max() * 1.01,  # Slightly above highest price
                fillcolor=color,
                opacity=0.7,
                layer="below",  # Place below candlesticks
                line_width=0,
            )
    
    # Add candlestick chart on top
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='Price'
        )
    )
    
    # Add legend for regimes
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color='rgba(255,0,0,0.8)'),
        name='Range'
    ))
    
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color='rgba(0,255,0,0.8)'),
        name='Trend'
    ))
    
    fig.update_layout(
        title='Price Chart with Market Regimes',
        xaxis_title='Date',
        yaxis_title='Price',
        height=800,
        showlegend=True
    )
    
    fig.show()

# Example usage:
plot_regimes_and_price(df_regimes, last_n_candles=335)